In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Import Graphs

In [2]:
import os
import networkx as nx

datasets = []

random_graph_directory = "./graphs/"
for root, dirs, files in os.walk(random_graph_directory):
    for filename in files:
        name_string = filename[:-4]
        graph_file_path = os.path.join(root, filename)

        G = nx.read_gml(graph_file_path)

        datasets.append(
            {
                "name": name_string,
                "graph": nx.relabel.convert_node_labels_to_integers(
                    G, first_label=0
                ),
            }
        )

## Define Solver Parameters

In [3]:
from solvers.dNNMIS_GPU_TAU import DNNMIS as V2
from solvers.KaMIS import ReduMIS
from solvers.ILPMIS import ILPMIS

solvers = [
    # {
    #     "name": "dNN V2 GPU",
    #     "class": V2,
    #     "params": {
    #         "learning_rate": 0.05,
    #         "selection_criteria": 0.5,
    #         "max_steps": 150000
    #     },
    # },
    # {
    #     "name": "dNN",
    #     "class": DNNMIS,
    #     "params": {
    #         "learning_rate": 0.001,
    #         "selection_criteria": 0.8,
    #         "max_steps": 25000,
    #     },
    # },
    #     {
    #     "name": "dNN w/SG5k",
    #     "class": DNNMIS,
    #     "params": {
    #         "learning_rate": 0.001,
    #         "selection_criteria": 0.65,
    #         "max_steps": 80000,
    #         "max_subgraph_steps": 5000,
    #     },
    # },
    {"name": "ReduMIS", "class": ReduMIS, "params": {"seed": 13}},
    # {"name": "ILP", "class": ILPMIS, "params": {"time_limit": 935}}
]

## Optional: Run New Benchmark

In [4]:
from copy import deepcopy


def benchmark(datasets, solvers):
    solutions = []

    stage = 0
    stages = len(solvers) * len(datasets)

    for solver in solvers:
        for dataset in datasets:
            solver_instance = solver["class"](dataset["graph"], solver["params"])
            solver_instance.solve()
            solution = {
                "solution_method": solver["name"],
                "dataset_name": dataset["name"],
                "data": deepcopy(solver_instance.solution),
                "time_taken": deepcopy(solver_instance.solution_time),
            }
            solutions.append(solution)
            del solver_instance
            stage += 1
            print(f"Completed {stage} / {stages}")

    return solutions

solutions = benchmark(datasets, solvers)

## Generate Results Table

In [ ]:
import pandas
import matplotlib.pyplot as plt
from itertools import combinations

dataset_index = {k: v for v, k in enumerate([dataset["name"] for dataset in datasets])}
datasets_solutions = [[] for i in range(len(datasets))]

def MIS_checker(MIS_list, G):
    pairs = list(combinations(MIS_list, 2))
    IS_CHECKER = True
    if len(MIS_list) > 1:
        for pair in pairs:
            if (pair[0], pair[1]) in G.edges or (pair[1], pair[0]) in G.edges:
                IS_CHECKER = False
                break
    return IS_CHECKER

table_data = []

for solution in solutions:
    dsi = dataset_index[solution["dataset_name"]]
    datasets_solutions[dsi].append(solution)

i = 0
for dataset_solutions in datasets_solutions:
    # IS CHECK
    is_check = []
    for solution in dataset_solutions:
        IS_set = solution["data"]["graph_mask"]
        print(IS_set)
        indices = [i for i, x in enumerate(IS_set) if x == 1]
        print(indices)
        subgraph = datasets[dataset_index[solution["dataset_name"]]]["graph"].subgraph(indices)
        subgraph = nx.Graph(subgraph)
        while len(subgraph) > 0:
            degrees = dict(subgraph.degree())
            max_degree_nodes = [node for node, degree in degrees.items() if degree == max(degrees.values())]
            
            if len(max_degree_nodes) == 0 or subgraph.degree(max_degree_nodes[0]) == 0:
                break  # No more nodes to remove or all remaining nodes have degree 0

            subgraph.remove_node(max_degree_nodes[0])
        nodes = list(subgraph)
        nodes.sort()
        print(nodes)
        # subgraph = datasets[dataset_index[solution["dataset_name"]]]["graph"].subgraph(
        #     IS_set
        # )
        # if len(subgraph.edges) > 0:
        #     plt.figure(i)
        #     plt.title(subgraph.edges)
        #     i += 1
        #     is_check.append(False)
        #     print(
        #         f"Non IS found using {solution['solution_method']} on {solution['dataset_name']}"
        #     )
        is_check.append(True)


    table_row = [dataset_solutions[0]['dataset_name']]

    table_row.extend([solution["data"]["size"] for solution in dataset_solutions])
    table_row.extend([solution["time_taken"] for solution in dataset_solutions])
    table_row.extend(is_check)

    table_data.append(table_row)

table_headers = ["Dataset Name"]

table_headers.extend([solver["name"] + " Solution Size" for solver in solvers])
table_headers.extend([solver["name"] + " Solution Time" for solver in solvers])
table_headers.extend([solver["name"] + " Solution IS" for solver in solvers])

table = pandas.DataFrame(table_data, columns=table_headers)
table
table.to_csv("results.csv")

[0 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 0 0]
[3, 4, 7, 9, 13]
[3, 4, 7, 9, 13]
[1 0 0 0 1 0 0 0 0 1 1 0]
[0, 4, 9, 10]
[0, 4, 9, 10]
[0 1 0 0 0 0 0 0 0 1 1 0 0 1]
[1, 9, 10, 13]
[1, 9, 10, 13]
[1 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0]
[0, 4, 6, 11, 14]
[0, 4, 6, 11, 14]
[1 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
[0, 1, 2, 3, 12, 16]
[0, 1, 2, 3, 12, 16]
[0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0]
[1, 4, 11, 13, 14]
[1, 4, 11, 13, 14]
[0 0 0 0 1 1 0 0 1 0 0 0 1 0]
[4, 5, 8, 12]
[4, 5, 8, 12]
[0 0 1 0 0 1 0 0 0 1]
[2, 5, 9]
[2, 5, 9]
[0 0 1 0 0 1 0 1 0 0 1 0 1 0]
[2, 5, 7, 10, 12]
[2, 5, 7, 10, 12]
[1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0, 2, 3, 4, 17]
[0, 2, 3, 4, 17]
[0 1 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0]
[1, 3, 4, 7, 13]
[1, 3, 4, 7, 13]
[0 0 0 0 1 0 0 0 1 0 1 1]
[4, 8, 10, 11]
[4, 8, 10, 11]
[0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0]
[1, 4, 10, 11, 13]
[1, 4, 10, 11, 13]
[0 0 0 0 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0]
[4, 6, 10, 11, 14]
[4, 6, 10, 11, 14]
[0 0 1 0 0 0 1 1 1 0]
[2, 6, 7, 8]
[2, 6, 7, 8]
[0 1 0 1 1 0 0

/tmp/ipykernel_167791/549185145.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


## Optional: Find Exact Sets
```{note}
Result is indexed by 0
```

In [ ]:
graph_of_interest = "GNM_n50_m613_seed6"

for dataset_solutions in datasets_solutions:
    for solution in dataset_solutions:
        if (solution["dataset_name"] == graph_of_interest):
            set_solution = []
            for idx, x in enumerate(solution["data"]["graph_mask"]):
                if x == 1:
                    set_solution.append(idx)
            print(solution["solution_method"],set_solution)

## Optional: Save Solutions to Pickle

In [ ]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

def read_object(filename):
    with open(filename, 'rb') as outp:  # Overwrites any existing file.
        return pickle.load(outp)

solutions = save_object(solutions, "solutions.pkl")